In [244]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import boto3

In [245]:
s3 = boto3.client('s3', 
                  aws_access_key_id = '', 
                  aws_secret_access_key = '')

bucketname = ''
file = 'train_kOBLwZA.csv'
s3.download_file(bucketname, file, '../Data/Train.csv')

In [246]:
# import redshift_connector

# """conn = redshift_connector.connect(host = hostname, database = databasename, user = username, password = password, port = 123456)"""

In [247]:
df = pd.read_csv("../Data/Train.csv")

In [248]:
df.sample(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1615,FDC03,8.575,reg,0.071958,Dairy,195.3794,OUT049,1999,Medium,Tier 1,Supermarket Type1,3511.4292
7403,FDL25,6.920,Regular,0.131458,Breakfast,90.5804,OUT018,2009,Medium,Tier 3,Supermarket Type2,183.7608
8021,NCN05,8.235,Low Fat,0.014519,Health and Hygiene,182.1950,OUT018,2009,Medium,Tier 3,Supermarket Type2,2380.2350
3000,FDK34,13.350,Low Fat,0.038605,Snack Foods,236.3564,OUT045,2002,NaN,Tier 2,Supermarket Type1,2145.2076
1106,FDT46,11.350,Low Fat,0.030870,Snack Foods,49.8008,OUT045,2002,NaN,Tier 2,Supermarket Type1,708.4112


In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [250]:
df.shape

(8523, 12)

In [251]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Item_Weight,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Visibility,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_MRP,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Establishment_Year,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Item_Outlet_Sales,8523.0,2181.288914,1706.499616,33.290,834.247400,1794.331000,3101.296400,13086.964800


In [252]:
df.describe(include='object').T

,count,unique,top,freq
Item_Identifier,8523,1559,FDW13,10
Item_Fat_Content,8523,5,Low Fat,5089
Item_Type,8523,16,Fruits and Vegetables,1232
Outlet_Identifier,8523,10,OUT027,935
Outlet_Size,6113,3,Medium,2793
Outlet_Location_Type,8523,3,Tier 3,3350
Outlet_Type,8523,4,Supermarket Type1,5577


In [253]:
cat_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

for col in cat_cols:
    print(f'The unique values in {col} are :')
    print(df[col].value_counts())
    print('-'*50)

The unique values in Item_Fat_Content are :
Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64
--------------------------------------------------
The unique values in Item_Type are :
Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64
--------------------------------------------------
The unique values in Outlet_Size are :
Outlet_Size
Medium    2793
Small     2388
High       932
Name: count, dtype: int64
--------------------------------------------------
The u

In [254]:
df.duplicated().sum()

0

In [255]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [256]:
df['Item_Weight'] = df['Item_Weight'].fillna(df.groupby('Item_Type')['Item_Weight'].transform('mean'))

In [257]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [258]:

outlet_cols = ['Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

df_filtered = df[outlet_cols][df['Outlet_Size'].notnull()]

#grouped = df_filtered.groupby(outlet_cols).count().reset_index()
grouped = df_filtered.groupby(outlet_cols).size().reset_index(name='Count')

grouped

,Outlet_Size,Outlet_Location_Type,Outlet_Type,Count
0,High,Tier 3,Supermarket Type1,932
1,Medium,Tier 1,Supermarket Type1,930
2,Medium,Tier 3,Supermarket Type2,928
3,Medium,Tier 3,Supermarket Type3,935
4,Small,Tier 1,Grocery Store,528
5,Small,Tier 1,Supermarket Type1,930
6,Small,Tier 2,Supermarket Type1,930


In [259]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [260]:
null_rows = df[(df['Outlet_Size'].isnull()) & 
               (df['Outlet_Location_Type']=='Tier 3') & 
               (df['Outlet_Type']=='Supermarket Type1')]

null_rows

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [261]:
null_rows = df[(df['Outlet_Size'].isnull()) & 
               (df['Outlet_Location_Type']=='Tier 2') & 
               (df['Outlet_Type']=='Supermarket Type1')]

null_rows

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8,FDH17,16.20,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.20,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350
25,NCD06,13.00,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080
33,FDO23,17.85,Low Fat,0.000000,Breads,93.1436,OUT045,2002,NaN,Tier 2,Supermarket Type1,2174.5028
46,NCP05,19.60,Low Fat,0.000000,Health and Hygiene,153.3024,OUT045,2002,NaN,Tier 2,Supermarket Type1,2428.8384
...,...,...,...,...,...,...,...,...,...,...,...,...
8501,FDW21,5.34,Regular,0.005998,Snack Foods,100.4358,OUT017,2007,NaN,Tier 2,Supermarket Type1,1508.0370
8502,NCH43,8.42,Low Fat,0.070712,Household,216.4192,OUT045,2002,NaN,Tier 2,Supermarket Type1,3020.0688
8508,FDW31,11.35,Regular,0.043246,Fruits and Vegetables,199.4742,OUT045,2002,NaN,Tier 2,Supermarket Type1,2587.9646
8514,FDA01,15.00,Regular,0.054489,Canned,57.5904,OUT045,2002,NaN,Tier 2,Supermarket Type1,468.7232


In [262]:
outlet_cols = ['Outlet_Identifier', 'Outlet_Size', 'Outlet_Type', 'Outlet_Location_Type', 'Outlet_Establishment_Year']

df_filtered = df[outlet_cols]

# Replace NaN values in Outlet_Size with a placeholder 'Unknown'
df_filtered['Outlet_Size'] = df_filtered['Outlet_Size'].fillna('Unknown')

grouped = df_filtered.groupby(outlet_cols).size().reset_index(name='Count')

grouped

,Outlet_Identifier,Outlet_Size,Outlet_Type,Outlet_Location_Type,Outlet_Establishment_Year,Count
0,OUT010,Unknown,Grocery Store,Tier 3,1998,555
1,OUT013,High,Supermarket Type1,Tier 3,1987,932
2,OUT017,Unknown,Supermarket Type1,Tier 2,2007,926
3,OUT018,Medium,Supermarket Type2,Tier 3,2009,928
4,OUT019,Small,Grocery Store,Tier 1,1985,528
5,OUT027,Medium,Supermarket Type3,Tier 3,1985,935
6,OUT035,Small,Supermarket Type1,Tier 2,2004,930
7,OUT045,Unknown,Supermarket Type1,Tier 2,2002,929
8,OUT046,Small,Supermarket Type1,Tier 1,1997,930
9,OUT049,Medium,Supermarket Type1,Tier 1,1999,930


In [263]:
df.loc[(df['Outlet_Size'].isnull()) & 
       (df['Outlet_Location_Type']=='Tier 2') & 
       (df['Outlet_Type']=='Supermarket Type1'),
       'Outlet_Size']='Small'


In [264]:
df.isnull().sum()

Item_Identifier                0
Item_Weight                    0
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  555
Outlet_Location_Type           0
Outlet_Type                    0
Item_Outlet_Sales              0
dtype: int64

In [265]:
outlet_cols = ['Outlet_Location_Type', 'Outlet_Type']

null_rows = df[outlet_cols][df['Outlet_Size'].isnull()]

grouped = null_rows.groupby(outlet_cols).size().reset_index(name='Count')

grouped

,Outlet_Location_Type,Outlet_Type,Count
0,Tier 3,Grocery Store,555


In [266]:
df.loc[(df['Outlet_Size'].isnull()),
       'Outlet_Size']='Small'

In [267]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [268]:
# Item_Fat_Content
# Low Fat    5089
# Regular    2889
# LF          316
# reg         117
# low fat     112

# df['Item_Fat_Content'].replace('LF', 'Low Fat')

# df.replace({'Item_Fat_Content':{'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'}}, inplace=True)

In [269]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('LF', 'Low Fat').replace('reg', 'Regular').replace('low fat', 'Low Fat')

In [270]:
df['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [271]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [272]:
df['Outlet_Size'].replace('High', 'Large', inplace=True)

df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Large,Tier 3,Supermarket Type1,994.7052


In [273]:
df['Outlet_Size'].value_counts()

Outlet_Size
Small     4798
Medium    2793
Large      932
Name: count, dtype: int64

In [274]:

df.groupby('Outlet_Size')['Outlet_Identifier'].agg({'unique', 'count'})

,unique,count
Outlet_Size,,
Large,[OUT013],932
Medium,"[OUT049, OUT018, OUT027]",2793
Small,"[OUT010, OUT045, OUT017, OUT046, OUT035, OUT019]",4798


In [275]:

df.groupby('Outlet_Size')['Outlet_Type'].agg({'unique', 'count'})

,unique,count
Outlet_Size,,
Large,[Supermarket Type1],932
Medium,"[Supermarket Type1, Supermarket Type2, Superma...",2793
Small,"[Grocery Store, Supermarket Type1]",4798


In [276]:
df.groupby('Outlet_Size')[['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Type']].agg({'unique'})

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Type
,unique,unique,unique
Outlet_Size,,,
Large,[OUT013],[1987],[Supermarket Type1]
Medium,"[OUT049, OUT018, OUT027]","[1999, 2009, 1985]","[Supermarket Type1, Supermarket Type2, Superma..."
Small,"[OUT010, OUT045, OUT017, OUT046, OUT035, OUT019]","[1998, 2002, 2007, 1997, 2004, 1985]","[Grocery Store, Supermarket Type1]"


In [277]:
outlet_cols = ['Outlet_Identifier', 'Outlet_Size', 'Outlet_Type', 'Outlet_Establishment_Year']

df_filtered = df[outlet_cols]

grouped = df_filtered.groupby(outlet_cols).size().reset_index(name='Count')

grouped

,Outlet_Identifier,Outlet_Size,Outlet_Type,Outlet_Establishment_Year,Count
0,OUT010,Small,Grocery Store,1998,555
1,OUT013,Large,Supermarket Type1,1987,932
2,OUT017,Small,Supermarket Type1,2007,926
3,OUT018,Medium,Supermarket Type2,2009,928
4,OUT019,Small,Grocery Store,1985,528
5,OUT027,Medium,Supermarket Type3,1985,935
6,OUT035,Small,Supermarket Type1,2004,930
7,OUT045,Small,Supermarket Type1,2002,929
8,OUT046,Small,Supermarket Type1,1997,930
9,OUT049,Medium,Supermarket Type1,1999,930


In [278]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Large,Tier 3,Supermarket Type1,994.7052


In [279]:
df.to_csv("../Data/cleaned_df.csv")